In [3]:
import tensorflow as tf

def inception_block(inputs, filters):
    tower_1 = tf.keras.layers.Conv2D(filters, (1, 1), padding='same', activation='relu')(inputs)
    tower_1 = tf.keras.layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(tower_1)

    tower_2 = tf.keras.layers.Conv2D(filters, (1, 1), padding='same', activation='relu')(inputs)
    tower_2 = tf.keras.layers.Conv2D(filters, (5, 5), padding='same', activation='relu')(tower_2)

    tower_3 = tf.keras.layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    tower_3 = tf.keras.layers.Conv2D(filters, (1, 1), padding='same', activation='relu')(tower_3)

    output = tf.keras.layers.concatenate([tower_1, tower_2, tower_3], axis=-1)
    return output

def inception_unet(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Downsampling
    conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer='glorot_normal')(inputs)
    conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = inception_block(pool1, 128)
    conv2 = tf.keras.layers.Conv2D(128, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv2)
    conv2 = tf.keras.layers.Conv2D(128, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = inception_block(pool2, 256)
    conv3 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv3)
    conv3 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv3)
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = inception_block(pool3, 512)
    conv4 = tf.keras.layers.Conv2D(512, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv4)
    conv4 = tf.keras.layers.Conv2D(512, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv4)
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bridge
    conv5 = inception_block(pool4, 1024)
    conv5 = tf.keras.layers.Conv2D(1024, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv5)
    conv5 = tf.keras.layers.Conv2D(1024, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv5)

    # Upsampling
    up6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
    up6 = tf.keras.layers.concatenate([up6, conv4], axis=-1)
    conv6 = inception_block(up6, 512)
    conv6 = tf.keras.layers.Conv2D(512, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv6)
    conv6 = tf.keras.layers.Conv2D(512, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv6)

    up7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = tf.keras.layers.concatenate([up7, conv3], axis=-1)
    conv7 = inception_block(up7, 256)
    conv7 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv7)
    conv7 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv7)

    up8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = tf.keras.layers.concatenate([up8, conv2], axis=-1)
    conv8 = inception_block(up8, 128)
    conv8 = tf.keras.layers.Conv2D(128, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv8)
    conv8 = tf.keras.layers.Conv2D(128, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv8)

    up9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = tf.keras.layers.concatenate([up9, conv1], axis=-1)
    conv9 = inception_block(up9, 64)
    conv9 = tf.keras.layers.Conv2D(64, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv9)
    conv9 = tf.keras.layers.Conv2D(64, (1, 1), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv9)

    # Output
    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv9)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model


In [2]:

# Define hyperparameters and other settings
input_shape = (256, 256, 1)
num_classes = 2
learning_rate = 0.001
num_epochs = 170

# Create the model
model = inception_unet(input_shape, num_classes)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_59 (Conv2D)             (None, 256, 256, 64  640         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_60 (Conv2D)             (None, 256, 256, 64  36928       ['conv2d_59[0][0]']              
                                )                                                           